# Projeto 1 - Ciência dos Dados

Nome: Giovanna Alves Papandrea Neves

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [119]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [120]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\User\Documents\Trabalho cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [121]:
dados = pd.read_excel('league of legends.xlsx', sheet_name='Treinamento')
dados.head()

,Classificacao,Treinamento
0,1,desafio da quarentena:\nfale 4 games que marca...
1,2,@pahssion agora poderemos unir league of legen...
2,1,desafio da quarentena:\n\ndiga 4 jogos que mar...
3,1,o league of legends levou ao extremo sua zoeir...
4,1,"o cara era meu namorado e nem sabia, a gente s..."


In [122]:
dados_teste = pd.read_excel('league of legends.xlsx', sheet_name='Teste')

#renomeando coluna de teste a fim de diferenciar classificação manual e da máquina.
dados_teste.rename(columns={'Classificação: 1':'Classificação'}, inplace=True) 
dados_teste.head()

,Classificacao,Teste
0,1,desafio da quarentena: \nfale 4 jogos que marc...
1,1,"rt @seimaineimi: 103,6 mil pessoas assistindo...."
2,1,desafio da quarentena: fale 4 jogos que marcar...
3,1,estou sendo tentada a aprender a jogar league ...
4,2,caralho que ódio de league of legends porra de...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

League of Legends é um jogo classificado como MOBA, e atualmente é o jogo mais popular do mundo, com mais de 100 Milhôes de usuários ativos todos os meses. Com isso, decidi classificar os tweets como positivos(P) ou negativos(N), para avaliar a opinião do público sobre o game. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [123]:
#Limpeza de Tweets
import re 

def cleanup(text):
    #Função para substituir os pontos por espaço
    
    punctuation = '[!\-.:?;•,]' #Pontuações
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed


Marca-se as classificações como variáveis qualitativas:


In [124]:
dados['Classificacao'] = dados['Classificacao'].astype('category')


In [125]:
dados['Classificacao'].cat.categories = ['P', 'N']

# Vendo o resultados
dados['Classificacao'].cat.categories

Index(['P', 'N'], dtype='object')

In [126]:
dados['Classificacao'].head(10)

0    P
1    N
2    P
3    P
4    P
5    N
6    P
7    P
8    P
9    P
Name: Classificacao, dtype: category
Categories (2, object): [P, N]

In [127]:
# P(P), P(N)
dados.Classificacao.value_counts(True)

P    0.883333
N    0.116667
Name: Classificacao, dtype: float64

In [128]:
priori = dados.Classificacao.value_counts(True, sort=False)
P_P = priori[0]
P_N = priori[1]
print(f'Probabilidade a priori de ser negativo {P_N:0.5g}')
print(f'Probabilidade a priori de ser positivo {P_P:0.5g}')

Probabilidade a priori de ser negativo 0.11667
Probabilidade a priori de ser positivo 0.88333


In [129]:

def Frequencia_Relativa(cat):
    #tira espaços
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    
    #Extra: remoção dos "http" para não enviezar o classificador.
    remove_http = re.sub(r'http\S+', '', tweet)
    text_clean = cleanup(remove_http)
    
    #split() e pd.Series()
    serie_dados = text_clean.split()
    freq_rel = pd.Series(serie_dados).value_counts(True)
    
    return freq_rel

In [130]:
# P(palabra|P)
Frequencia_Relativa('P').head()

of         0.043554
legends    0.036562
league     0.036562
4          0.035688
que        0.025055
dtype: float64

In [131]:

# P(palavra|N)
Frequencia_Relativa('N').head()

of         0.063406
legends    0.063406
league     0.063406
de         0.038043
não        0.021739
dtype: float64

In [132]:

dados_P = dados[dados.Classificacao == 'P'].copy()
dados_P.head(5)

,Classificacao,Treinamento
0,P,desafio da quarentena:\nfale 4 games que marca...
2,P,desafio da quarentena:\n\ndiga 4 jogos que mar...
3,P,o league of legends levou ao extremo sua zoeir...
4,P,"o cara era meu namorado e nem sabia, a gente s..."
6,P,.@pedrooneves7 ranked flex de league of legend...


In [133]:
dados_N = dados[dados.Classificacao == 'N'].copy()
dados_N.head(5)

,Classificacao,Treinamento
1,N,@pahssion agora poderemos unir league of legen...
5,N,já estou cansado de você propaganda de league ...
16,N,o que precisa fazer para ganhar uma partida de...
20,N,rt @brinydelaet: tentando escapar de loucuras ...
25,N,@prodwiththerod meu league of legends não está...


In [134]:
def Frequencia_Absoluta(cat):
    
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+', '', tweet)
    
    text_clean = cleanup(remove_http)
    serie_dados = text_clean.split()
    freq_abs = pd.Series(serie_dados).value_counts(True)
    
    return freq_abs

In [135]:

Frequencia_Absoluta('P').head()

of         0.043554
legends    0.036562
league     0.036562
4          0.035688
que        0.025055
dtype: float64

In [136]:
Frequencia_Absoluta('N').head()

of         0.063406
legends    0.063406
league     0.063406
de         0.038043
não        0.021739
dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**